In [ ]:
!pip install python-dotenv

In [ ]:
import google.generativeai as genai

from dotenv import load_dotenv
load_dotenv()

import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
import pandas as pd

df = pd.read_csv("football_questions.csv", sep=";")
df.head(10)


,id,question,answer,ground_truth_label,ground_truth_reasoning
0,1,Koji klub je osvojio Premier Ligu u sezoni 202...,Liverpool,Pass,Tocan odgovor je Liverpool
1,2,Koji tim je zavrsio na drugom mjestu u Premier...,Manchester United,Fail,"Netocno, tocan odgovor je Arsenal"
2,3,Koja tri kluba su ispala iz Premier Lige 2024/...,"Leicester, Southampton",Fail,"Nepotpun odgovor, treba 3 kluba"
3,4,Koji klub je zavrsio na cetvrtom mjestu i osig...,Chelsea,Pass,Tocan odgovor je Chelsea
4,5,Koliko je bodova imao prvak Premier Lige na kr...,82,Fail,Netocno. Imao je 84
5,6,Tko je bio najbolji strijelac Premier Lige u s...,Salah,Pass,"Tocno, Salah je najbolji strijelac"
6,7,Koliko je golova postigao najbolji strijelac?,29,Pass,"Tocno, zabio je 29 golova"
7,8,Koji igrac je imao najvise asistencija u sezoni?,Mohammed S.,Pass,"Tocno, iako je napisano skraceno"
8,9,Tko je bio golman godine u Premier Ligi 2024/2...,D. Raya,Pass,"Tocno, golman sezone bio je David Raya"
9,10,Koji igrac je osvojio nagradu za igraca sezone?,M. Salah,Pass,"Tocno, Salah je osvojio nagradu za igraca sezone"


In [ ]:
import time
from tqdm import tqdm

prompt_template = """
Tvoj zadatak:
- Procijeni korisnikov odgovor prema TOČNOM odgovoru u JSON-u.
- DOZVOLJENE varijante: inicijali, nadimci, male tipfelere.

PRIMJERI EVALUACIJE (Ground Truth):
{{"id": 1, "question": "Koji klub je osvojio Premier Ligu u sezoni 2024/2025?", "answer": "Liverpool"}},
{{"id": 2, "question": "Koji tim je zavrsio na drugom mjestu u Premier Ligi 2024/2025?", "answer": "Arsenal"}},
{{"id": 3, "question": "Koja tri kluba su ispala iz Premier Lige 2024/2025?", "answer": "Leicester, Southampton, Ipswich"}},
{{"id": 4, "question": "Koji klub je zavrsio na cetvrtom mjestu i osigurao Ligu prvaka?", "answer": "Chelsea"}},
{{"id": 5, "question": "Koliko je bodova imao prvak Premier Lige na kraju sezone?", "answer": "84"}},
{{"id": 6, "question": "Tko je bio najbolji strijelac Premier Lige u sezoni 2024/2025?", "answer": "Mohammed Salah"}},
{{"id": 7, "question": "Koliko je golova postigao najbolji strijelac?", "answer": "29"}},
{{"id": 8, "question": "Koji igrac je imao najvise asistencija u sezoni?", "answer": "Mohammed Salah"}},
{{"id": 9, "question": "Tko je bio golman godine u Premier Ligi 2024/2025?", "answer": "David Raya"}},
{{"id": 10, "question": "Koji igrac je osvojio nagradu za igraca sezone?", "answer": "Mohammed Salah"}},
{{"id": 11, "question": "Koji mladi igrac (ispod 23 godine) se najvise isticao?", "answer": "Gravenberch"}},
{{"id": 12, "question": "Koji je bio rezultat north London derbija izmedu Arsenala i Tottenhama u prvom krugu?", "answer": "Arsenal je pobjedio 1-0"}},
{{"id": 13, "question": "Koji tim je imao najdulju pobjednicku seriju u sezoni?", "answer": "Newcastle"}},
{{"id": 14, "question": "Koja utakmica je imala najvise golova u sezoni?", "answer": "Tottenham - Liverpool (9 golova)"}},
{{"id": 15, "question": "Koji je trener bio otpusten prvi u sezoni 2024/2025?", "answer": "Erik Ten Hag"}},
{{"id": 16, "question": "Koji novi trener je preuzeo Manchester United tijekom sezone?", "answer": "Ruben Amorim"}},
{{"id": 17, "question": "Kolika je bila razlika u bodovima izmedu prvog i drugog mjesta?", "answer": "10"}},
{{"id": 18, "question": "Koji tim je imao najbolju obranu (najmanje primljenih golova)?", "answer": "Arsenal"}},
{{"id": 19, "question": "Koji tim je postigao najvise golova u sezoni?", "answer": "Liverpool"}},
{{"id": 20, "question": "Koji klub je imao najveci transfer u zimskom prijelaznom roku?", "answer": "Al Nassr"}}

Format odgovora:
Pass/Fail
kratko objašnjenje jedne rečenice

Pitanje: {question}
Odgovor: {answer}
"""

model = genai.GenerativeModel("gemini-2.5-flash")   # možeš i "gemini-2.0-flash" ili "gemini-1.5-pro"

def gemini_evaluator(question, answer, sleep=0.4):
    prompt = prompt_template.format(question=question, answer=answer)

    response = model.generate_content(
        prompt,
        generation_config={"temperature": 0}
    )

    text = response.text.strip()

    # Parsiranje
    if text.lower().startswith("pass"):
        label = "pass"
        reason = text[5:].strip(" :")
    elif text.lower().startswith("fail"):
        label = "fail"
        reason = text[5:].strip(" :")
    else:
        # fallback kad output nije formatiran
        parts = text.split(":", 1)
        if len(parts) == 2:
            label = parts[0].strip().lower()
            reason = parts[1].strip()
        else:
            label = "fail"
            reason = text

    time.sleep(sleep)
    return label, reason

In [ ]:
preds = []
reasons = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    label, reason = gemini_evaluator(row['question'], row['answer'])
    preds.append(label)
    reasons.append(reason)

df['eval_label'] = preds
df['eval_reason'] = reasons


100%|██████████| 20/20 [00:53<00:00,  2.69s/it]


In [ ]:
df.head(20)

,id,question,answer,ground_truth_label,ground_truth_reasoning,eval_label,eval_reason
0,1,Koji klub je osvojio Premier Ligu u sezoni 202...,Liverpool,Pass,Tocan odgovor je Liverpool,pass,Odgovor korisnika točno odgovara točnom odgovoru.
1,2,Koji tim je zavrsio na drugom mjestu u Premier...,Manchester United,Fail,"Netocno, tocan odgovor je Arsenal",fail,Točan odgovor je Arsenal.
2,3,Koja tri kluba su ispala iz Premier Lige 2024/...,"Leicester, Southampton",Fail,"Nepotpun odgovor, treba 3 kluba",fail,Odgovor je nepotpun jer nedostaje jedan od tri...
3,4,Koji klub je zavrsio na cetvrtom mjestu i osig...,Chelsea,Pass,Tocan odgovor je Chelsea,pass,Odgovor je točan i podudara se s očekivanim od...
4,5,Koliko je bodova imao prvak Premier Lige na kr...,82,Fail,Netocno. Imao je 84,fail,Korisnikov odgovor (82) se ne podudara s točni...
5,6,Tko je bio najbolji strijelac Premier Lige u s...,Salah,Pass,"Tocno, Salah je najbolji strijelac",pass,"Odgovor ""Salah"" je prihvatljiv kao nadimak za ..."
6,7,Koliko je golova postigao najbolji strijelac?,29,Pass,"Tocno, zabio je 29 golova",pass,Odgovor točno odgovara očekivanom broju golova.
7,8,Koji igrac je imao najvise asistencija u sezoni?,Mohammed S.,Pass,"Tocno, iako je napisano skraceno",pass,Odgovor je prihvatljiv jer koristi inicijale z...
8,9,Tko je bio golman godine u Premier Ligi 2024/2...,D. Raya,Pass,"Tocno, golman sezone bio je David Raya",pass,"Inicijali ""D. Raya"" su prihvatljiva varijanta ..."
9,10,Koji igrac je osvojio nagradu za igraca sezone?,M. Salah,Pass,"Tocno, Salah je osvojio nagradu za igraca sezone",pass,"Korisnikov odgovor ""M. Salah"" je prihvatljiva ..."


In [ ]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report, confusion_matrix

# binarno mapiranje
to_bin = {'pass': 1, 'fail': 0}

y_true = df['ground_truth_label'].str.lower().map(to_bin)
y_pred = df['eval_label'].map(to_bin)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Cohen's kappa:", cohen_kappa_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred, target_names=["Fail","Pass"]))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))


Accuracy: 1.0
Cohen's kappa: 1.0

Classification report:
               precision    recall  f1-score   support

        Fail       1.00      1.00      1.00         5
        Pass       1.00      1.00      1.00        15

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20


Confusion matrix:
 [[ 5  0]
 [ 0 15]]
